# 对算法的运行结果记录进行汇总，输出最终结果


- 小系统结果汇总


In [1]:
import numpy as np
import pandas as pd
from codehub_mcapplus import Common,Project,Case
import math
import copy
import time
import os
from collections import defaultdict

首先读取各个系统的pandas,以系统名字为key保存为字典

In [23]:
pattern = ['g','f']
nk = [[2,7],[3,7],[2,8],[3,8],[3,9],[4,9],[3,10],[4,10]]
condition = ['less']
namelist_small = []

for n in nk:
    for p in pattern:
        for c in condition:
            namelist_small.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
# 路径
path_small = '.\\result\\now\\smallsystem\\'
# 文件读取
df_dict = defaultdict(list)
Seednum = 1
for name in namelist_small:
    filepath = path_small + name + '_seed_'+ str(Seednum) + '.csv'
    df = pd.read_csv(filepath,index_col=0,header=None)
    df.columns = pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']
    ])
    df = df.loc[:,(slice(None),['ssr','processtime','enumerationprotime'])]
    df_dict[name] = df

In [24]:
df_dict[namelist_small[0]]

low                                high                                 \
   ssr processtime enumerationprotime  ssr processtime enumerationprotime   
0                                                                           
0  1.0    0.171875           0.046875  1.0     0.15625           0.046875   

  arbitrary                                 
        ssr processtime enumerationprotime  
0                                           
0       1.0     0.15625             0.0625

In [25]:
summary = pd.DataFrame(index = namelist_small,
                      columns= pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['mssr','time1','time2']
    ]))
summary

low             high             arbitrary            
            mssr time1 time2 mssr time1 time2      mssr time1 time2
g_2_7_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_2_7_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_7_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_7_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_2_8_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_2_8_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_8_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_8_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_9_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_9_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_4_9_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_4_9_less   NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_3_10_less  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_3_10_less  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
g_4_10_less  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN
f_4_10_less  NaN   NaN   NaN  NaN   NaN   NaN       NaN   NaN   NaN

In [26]:
for name in namelist_small:
    df = df_dict[name].sum(axis=0)
    summary.loc[name,:] = np.array(df)
summary.loc[:,(slice(None),'mssr')] = summary.loc[:,(slice(None),'mssr')] / Seednum
summary

low                          high                      \
                 mssr     time1     time2      mssr     time1     time2   
g_2_7_less          1  0.171875  0.046875         1   0.15625  0.046875   
f_2_7_less          1  0.265625  0.046875         1   0.15625  0.046875   
g_3_7_less   0.852277   0.15625  0.046875  0.621755  0.171875  0.046875   
f_3_7_less          1  0.234375  0.046875   0.77337   0.15625    0.0625   
g_2_8_less          1    0.1875   0.40625         1  0.234375   0.40625   
f_2_8_less          1  0.203125   0.40625         1    0.1875   0.40625   
g_3_8_less          1    0.3125  0.390625         1   0.21875     0.375   
f_3_8_less          1   0.21875   0.40625         1  0.203125   0.46875   
g_3_9_less          1      0.25   3.65625  0.971028   0.21875   3.64062   
f_3_9_less          1      0.25   3.89062  0.991507  0.234375   3.67188   
g_4_9_less   0.966887  0.265625    3.8125         1   0.21875   3.84375   
f_4_9_less          1   0.40625   3.85938  0.789778  0.234375   3.65625   
g_3_10_less         1  0.328125   37.3438         1   0.28125   35.8906   
f_3_10_less         1    0.3125   36.4531         1    0.3125   37.6719   
g_4_10_less  0.931603    0.3125   37.4531         1  0.390625   36.4844   
f_4_10_less         1  0.390625   36.1094  0.970093  0.296875   37.2812   

            arbitrary                      
                 mssr     time1     time2  
g_2_7_less          1   0.15625    0.0625  
f_2_7_less          1  0.265625  0.046875  
g_3_7_less   0.806222  0.171875  0.046875  
f_3_7_less   0.756234  0.203125  0.046875  
g_2_8_less          1  0.328125  0.421875  
f_2_8_less          1   0.21875  0.390625  
g_3_8_less          1  0.203125   0.40625  
f_3_8_less          1  0.203125   0.40625  
g_3_9_less          1  0.265625   3.78125  
f_3_9_less          1      0.25      3.75  
g_4_9_less   0.982479  0.296875   3.95312  
f_4_9_less   0.799315  0.234375    3.5625  
g_3_10_less         1  0.390625   37.9219  
f_3_10_less  0.988299  0.546875   38.4531  
g_4_10_less  0.906375  0.359375   36.2344  
f_4_10_less  0.997564    0.3125    38.375

- 大系统结果汇总

In [ ]:
pattern = ['g','f']
nk = [[4,20],[5,20],[10,20],[12,30],[15,30],[20,30],[22,40],[25,40],[30,40],[32,50],[35,50],[40,50]]
condition = ['less']
namelist_large = []
for p in pattern:
    for n in nk:
        for c in condition:
            namelist_large.append(p + '_' + str(n[0]) + '_' + str(n[1]) + '_' + c)
print(namelist_large)
# 路径
path_large = '.\\result\\now\\largesystem\\'
# 文件读取
df_dict = defaultdict(list)
Seednum = 10
for name in namelist_large:
    filepath = path_large + name + '_seed_'+ str(Seednum) + '.csv'
    df = pd.read_csv(filepath,index_col=0,header=None)
    df.columns = pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['ssr','sys','design','max_sys','best_design','min_sys','worst_design','processtime','time','enumerationprotime','enumerationtime']
    ])
    df = df.loc[:,(slice(None),['ssr','processtime','enumerationprotime'])]
    df_dict[name] = df

In [ ]:
df_dict[namelist_large[0]]

In [ ]:
summary = pd.DataFrame(index = namelist_large,
                      columns= pd.MultiIndex.from_product([
        ['low','high','arbitrary'],
        ['mssr','time1','time2']
    ]))
summary

In [ ]:
for name in namelist_large:
    df = df_dict[name].sum(axis=0)
    summary.loc[name,:] = np.array(df)
summary.loc[:,(slice(None),'mssr')] = summary.loc[:,(slice(None),'mssr')] / Seednum
summary